In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 771.5 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 102.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 91.8 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 120.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 26.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 KB 53.2 MB/s eta 0:00:00


In [54]:
%%writefile multigpu_weatheraus.py

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datautils import MyTrainDataset

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import os
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
from torch.utils.data import TensorDataset, DataLoader

def ddp_setup(rank, world_size):
    """
    Args:
        rank: Unique identifier of each process
        world_size: Total number of processes
    """
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"
    init_process_group(backend="nccl", rank=rank, world_size=world_size)
    torch.cuda.set_device(rank)

class Trainer:
    def __init__(
        self,
        model: torch.nn.Module,
        train_data: DataLoader,
        optimizer: torch.optim.Optimizer,
        gpu_id: int,
        save_every: int,
    ) -> None:
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.optimizer = optimizer
        self.save_every = save_every
        self.model = DDP(model, device_ids=[gpu_id])

    def _run_batch(self, source, targets):
        self.optimizer.zero_grad()
        output = self.model(source)
        loss = F.cross_entropy(output, targets)
        loss.backward()
        self.optimizer.step()

    def _run_epoch(self, epoch):
        b_sz = len(next(iter(self.train_data))[0])
        print(f"[GPU{self.gpu_id}] Epoch {epoch} | Batchsize: {b_sz} | Steps: {len(self.train_data)}")
        self.train_data.sampler.set_epoch(epoch)
        for source, targets in self.train_data:
            source = source.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            self._run_batch(source, targets)

    def _save_checkpoint(self, epoch):
        ckp = self.model.module.state_dict()
        PATH = "checkpoint_multigpu4.pt"
        torch.save(ckp, PATH)
        print(f"Epoch {epoch} | Training checkpoint saved at {PATH}")

    def train(self, max_epochs: int):
        for epoch in range(max_epochs):
            self._run_epoch(epoch)
            if self.gpu_id == 0 and epoch % self.save_every == 0:
                self._save_checkpoint(epoch)
                print("self.save_every" ,  self.save_every)
                print('save at epoch' , epoch)

class Net(nn.Module):    # class Net will be the subclass of torch.nn.Module ie Class Net --EXTENDS--> Class nn.Module
    def __init__(self, n_features):    # initialize the layers you want to use in this function/method
        super(Net, self).__init__()    # call to init method of superclass ie nn.Module
        self.fc1 = nn.Linear(n_features, 15)    # Input Layer of n_features input nodes to 5 outputs
        self.fc2 = nn.Linear(15, 3)             # 1st Hidden Layer of 5 nodes to 3 outputs
        self.fc3 = nn.Linear(3, 1)             # 2st Hidden Layer of 3 nodes to 1 output
    
    def forward(self, x):              # Feed Forward
        x = F.relu(self.fc1(x))        # torch.nn.functional.relu() ie a Activation Function 
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x)) 
    
def load_train_objs():
    # train_set = MyTrainDataset(2048)  # load your dataset
    df = pd.read_csv("weatherAUS.csv")
    df = df[['Rainfall','Humidity3pm','RainToday','Pressure9am','RainTomorrow']]
    df = df.dropna(how = 'any')
    df.shape

    df.RainToday[df.RainToday == 'Yes'] = 1 
    df.RainToday[df.RainToday == 'No'] = 0
    df.RainToday = pd.to_numeric(df.RainToday)
    df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
    df.RainTomorrow[df.RainTomorrow == 'No'] = 0
    df.RainTomorrow = pd.to_numeric(df.RainTomorrow)
    df.info()

    Y, X = df[['RainTomorrow']], df.drop('RainTomorrow', axis = 1, inplace = False)

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=2)


    # input must be numeric
    Xtrain = torch.from_numpy(Xtrain.to_numpy()).float()
    Xtest = torch.from_numpy(Xtest.to_numpy()).float()
    Ytrain = torch.from_numpy(Ytrain.to_numpy()).float()
    Ytest = torch.squeeze(torch.from_numpy(Ytest.to_numpy()).float())    

    # Create TensorDataset
    train_set = TensorDataset(Xtrain, Ytrain)
    test_dataset = TensorDataset(Xtest, Ytest)

    # model = torch.nn.Linear(20, 1)  # load your model
    model = Net(Xtrain.shape[1])
    
    
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(),  lr = 10 **(-4))
    return train_set, model, optimizer


def prepare_dataloader(dataset: Dataset, batch_size: int):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        sampler=DistributedSampler(dataset)
    )


def main(rank: int, world_size: int, save_every: int, total_epochs: int, batch_size: int):
    ddp_setup(rank, world_size)
    dataset, model, optimizer = load_train_objs()
    train_data = prepare_dataloader(dataset, batch_size)
    trainer = Trainer(model, train_data, optimizer, rank, save_every)
    trainer.train(total_epochs)
    destroy_process_group()


if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description='simple distributed training job')
    parser.add_argument('total_epochs', type=int, help='Total epochs to train the model')
    parser.add_argument('save_every', type=int, help='How often to save a snapshot')
    parser.add_argument('--batch_size', default=32, type=int, help='Input batch size on each device (default: 32)')
    args = parser.parse_args()
    
    world_size = torch.cuda.device_count()
    mp.spawn(main, args=(world_size, args.save_every, args.total_epochs, args.batch_size), nprocs=world_size)

Overwriting multigpu_weatheraus.py


In [55]:
%%time
!python multigpu_weatheraus.py 81 10 --batch_size 64

[W socket.cpp:663] [c10d] The client socket has failed to connect to [localhost]:12355 (errno: 99 - Cannot assign requested address).
/dli/task/multigpu_weatheraus.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainToday[df.RainToday == 'No'] = 0
/dli/task/multigpu_weatheraus.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainToday[df.RainToday == 'No'] = 0
/dli/task/multigpu_weatheraus.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

# Test

In [56]:
import torch, glob
import torch.optim as optim
import os
import warnings
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
from torch.utils.data import TensorDataset, DataLoader

df = pd.read_csv("weatherAUS.csv")
df = df[['Rainfall','Humidity3pm','RainToday','Pressure9am','RainTomorrow']]
df = df.dropna(how = 'any')
df.shape

df.RainToday[df.RainToday == 'Yes'] = 1 
df.RainToday[df.RainToday == 'No'] = 0
df.RainToday = pd.to_numeric(df.RainToday)
df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
df.RainTomorrow[df.RainTomorrow == 'No'] = 0
df.RainTomorrow = pd.to_numeric(df.RainTomorrow)
df.info()

Y, X = df[['RainTomorrow']], df.drop('RainTomorrow', axis = 1, inplace = False)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=2)

Xtest = torch.from_numpy(Xtest.to_numpy()).float()

class Net(nn.Module):    # class Net will be the subclass of torch.nn.Module ie Class Net --EXTENDS--> Class nn.Module
    def __init__(self, n_features):    # initialize the layers you want to use in this function/method
        super(Net, self).__init__()    # call to init method of superclass ie nn.Module
        self.fc1 = nn.Linear(n_features, 15)    # Input Layer of n_features input nodes to 5 outputs
        self.fc2 = nn.Linear(15, 3)             # 1st Hidden Layer of 5 nodes to 3 outputs
        self.fc3 = nn.Linear(3, 1)             # 2st Hidden Layer of 3 nodes to 1 output
    
    def forward(self, x):              # Feed Forward
        x = F.relu(self.fc1(x))        # torch.nn.functional.relu() ie a Activation Function 
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x)) 
files = []
files += glob.glob('checkpoint*.pt')
# files += glob.glob('my_model*.pth')

for model_file in files:
    
    net = Net(Xtrain.shape[1])
    state_dict = torch.load(model_file)

    # Remove the "module." prefix from the keys in the state dictionary
    state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

    # Load the modified state dictionary
    net.load_state_dict(state_dict)
    net.eval()


    classes = ['No rain', 'Rain']

    y_test = torch.squeeze(torch.from_numpy(Ytest.to_numpy()).float()) 

    y_pred = net(Xtest)
    y_pred
    # Convert the probabilities to binary classes ie (1 or 0) and (True or False) by the help of threshold values
    y_pred = y_pred.ge(.5).view(-1).cpu() # test on cpu ?? yes
    y_pred

    y_test = y_test.cpu()
    from sklearn.metrics import classification_report
    print(f'{model_file}')
    print(classification_report(y_test, y_pred, target_names = classes))  
    del net

/tmp/ipykernel_158/2113590087.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainToday[df.RainToday == 'No'] = 0
/tmp/ipykernel_158/2113590087.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
/tmp/ipykernel_158/2113590087.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'No'] = 0
/opt/conda/envs/rapids/lib/python3.8/site-packages/sklearn/m

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124689 entries, 0 to 145458
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Rainfall      124689 non-null  float64
 1   Humidity3pm   124689 non-null  float64
 2   RainToday     124689 non-null  int64  
 3   Pressure9am   124689 non-null  float64
 4   RainTomorrow  124689 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 5.7 MB
checkpoint_multigpu3.pt
              precision    recall  f1-score   support

     No rain       0.00      0.00      0.00     19361
        Rain       0.22      1.00      0.37      5577

    accuracy                           0.22     24938
   macro avg       0.11      0.50      0.18     24938
weighted avg       0.05      0.22      0.08     24938

checkpoint_singleGPU.pt
              precision    recall  f1-score   support

     No rain       0.78      1.00      0.87     19361
        Rain       0.00      0.00      0.00     

/opt/conda/envs/rapids/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/rapids/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/rapids/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/rapids/lib/pyth

In [19]:
%%writefile signlegpu_weatheraus.py
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datautils import MyTrainDataset
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datautils import MyTrainDataset

import torch.multiprocessing as mp
from torch.utils.data.distributed import DistributedSampler
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
import os
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
from torch.utils.data import TensorDataset, DataLoader

class Trainer:
    def __init__(
        self,
        model: torch.nn.Module,
        train_data: DataLoader,
        optimizer: torch.optim.Optimizer,
        gpu_id: int,
        save_every: int, 
    ) -> None:
        self.gpu_id = gpu_id
        self.model = model.to(gpu_id)
        self.train_data = train_data
        self.optimizer = optimizer
        self.save_every = save_every

    def _run_batch(self, source, targets):
        self.optimizer.zero_grad()
        output = self.model(source)
        loss = F.cross_entropy(output, targets)
        loss.backward()
        self.optimizer.step()

    def _run_epoch(self, epoch):
        b_sz = len(next(iter(self.train_data))[0])
        print(f"[GPU{self.gpu_id}] Epoch {epoch} | Batchsize: {b_sz} | Steps: {len(self.train_data)}")
        for source, targets in self.train_data:
            source = source.to(self.gpu_id)
            targets = targets.to(self.gpu_id)
            self._run_batch(source, targets)

    def _save_checkpoint(self, epoch):
        ckp = self.model.state_dict()
        PATH = "checkpoint_singleGPU.pt" #here ? yes
        torch.save(ckp, PATH)
        print(f"Epoch {epoch} | Training checkpoint saved at {PATH}")

    def train(self, max_epochs: int):
        for epoch in range(max_epochs):
            self._run_epoch(epoch)
            if epoch % self.save_every == 0:
                self._save_checkpoint(epoch)


class Net(nn.Module):    # class Net will be the subclass of torch.nn.Module ie Class Net --EXTENDS--> Class nn.Module
    def __init__(self, n_features):    # initialize the layers you want to use in this function/method
        super(Net, self).__init__()    # call to init method of superclass ie nn.Module
        self.fc1 = nn.Linear(n_features, 15)    # Input Layer of n_features input nodes to 5 outputs
        self.fc2 = nn.Linear(15, 3)             # 1st Hidden Layer of 5 nodes to 3 outputs
        self.fc3 = nn.Linear(3, 1)             # 2st Hidden Layer of 3 nodes to 1 output
    
    def forward(self, x):              # Feed Forward
        x = F.relu(self.fc1(x))        # torch.nn.functional.relu() ie a Activation Function 
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x)) 
    
def load_train_objs():
     # train_set = MyTrainDataset(2048)  # load your dataset
    df = pd.read_csv("weatherAUS.csv")
    df = df[['Rainfall','Humidity3pm','RainToday','Pressure9am','RainTomorrow']]
    df = df.dropna(how = 'any')
    df.shape

    df.RainToday[df.RainToday == 'Yes'] = 1 
    df.RainToday[df.RainToday == 'No'] = 0
    df.RainToday = pd.to_numeric(df.RainToday)
    df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
    df.RainTomorrow[df.RainTomorrow == 'No'] = 0
    df.RainTomorrow = pd.to_numeric(df.RainTomorrow)
    df.info()

    Y, X = df[['RainTomorrow']], df.drop('RainTomorrow', axis = 1, inplace = False)

    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=2)


    # input must be numeric
    Xtrain = torch.from_numpy(Xtrain.to_numpy()).float()
    Xtest = torch.from_numpy(Xtest.to_numpy()).float()
    Ytrain = torch.from_numpy(Ytrain.to_numpy()).float()
    Ytest = torch.squeeze(torch.from_numpy(Ytest.to_numpy()).float())    

    # Create TensorDataset
    train_set = TensorDataset(Xtrain, Ytrain)
    test_dataset = TensorDataset(Xtest, Ytest)

    # model = torch.nn.Linear(20, 1)  # load your model
    model = Net(Xtrain.shape[1])
    
    
    # optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
    optimizer = torch.optim.Adam(model.parameters(),  lr = 10 **(-4))
    return train_set, model, optimizer


def prepare_dataloader(dataset: Dataset, batch_size: int):
    return DataLoader(
        dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=True
    )


def main(device, total_epochs, save_every, batch_size):
    dataset, model, optimizer = load_train_objs()
    train_data = prepare_dataloader(dataset, batch_size)
    trainer = Trainer(model, train_data, optimizer, device, save_every)
    trainer.train(total_epochs)


if __name__ == "__main__":
    import argparse
    parser = argparse.ArgumentParser(description='simple distributed training job')
    parser.add_argument('total_epochs', type=int, help='Total epochs to train the model')
    parser.add_argument('save_every', type=int, help='How often to save a snapshot')
    parser.add_argument('--batch_size', default=32, type=int, help='Input batch size on each device (default: 32)')
    args = parser.parse_args()
    
    device = 0  # shorthand for cuda:0
    main(device, args.total_epochs, args.save_every, args.batch_size)

Writing signlegpu_weatheraus.py


In [20]:
%%time
!python signlegpu_weatheraus.py 81 10 --batch_size 128

signlegpu_weatheraus.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainToday[df.RainToday == 'No'] = 0
signlegpu_weatheraus.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
signlegpu_weatheraus.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'No'] = 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 124689 entries, 0 to 145458
Data colu

# Test

In [22]:
import torch, glob
import torch.optim as optim
import os
import warnings
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
from torch.utils.data import TensorDataset, DataLoader

df = pd.read_csv("weatherAUS.csv")
df = df[['Rainfall','Humidity3pm','RainToday','Pressure9am','RainTomorrow']]
df = df.dropna(how = 'any')
df.shape

df.RainToday[df.RainToday == 'Yes'] = 1 
df.RainToday[df.RainToday == 'No'] = 0
df.RainToday = pd.to_numeric(df.RainToday)
df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
df.RainTomorrow[df.RainTomorrow == 'No'] = 0
df.RainTomorrow = pd.to_numeric(df.RainTomorrow)
df.info()

Y, X = df[['RainTomorrow']], df.drop('RainTomorrow', axis = 1, inplace = False)
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=2)

Xtest = torch.from_numpy(Xtest.to_numpy()).float()

class Net(nn.Module):    # class Net will be the subclass of torch.nn.Module ie Class Net --EXTENDS--> Class nn.Module
    def __init__(self, n_features):    # initialize the layers you want to use in this function/method
        super(Net, self).__init__()    # call to init method of superclass ie nn.Module
        self.fc1 = nn.Linear(n_features, 15)    # Input Layer of n_features input nodes to 5 outputs
        self.fc2 = nn.Linear(15, 3)             # 1st Hidden Layer of 5 nodes to 3 outputs
        self.fc3 = nn.Linear(3, 1)             # 2st Hidden Layer of 3 nodes to 1 output
    
    def forward(self, x):              # Feed Forward
        x = F.relu(self.fc1(x))        # torch.nn.functional.relu() ie a Activation Function 
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x)) 
files = []
files += glob.glob('checkpoint_singleGPU*.pt') # here i think there could be wrong , not wrong
# files += glob.glob('my_model*.pth')

for model_file in files:
    
    net = Net(Xtrain.shape[1])
    state_dict = torch.load(model_file)

    # Remove the "module." prefix from the keys in the state dictionary
    state_dict = {k.replace("module.", ""): v for k, v in state_dict.items()}

    # Load the modified state dictionary
    net.load_state_dict(state_dict)
    net.eval()

    classes = ['No rain', 'Rain']

    y_test = torch.squeeze(torch.from_numpy(Ytest.to_numpy()).float()) 

    y_pred = net(Xtest)
    y_pred
    # Convert the probabilities to binary classes ie (1 or 0) and (True or False) by the help of threshold values
    y_pred = y_pred.ge(.5).view(-1).cpu() # test on cpu ?? yes
    y_pred

    y_test = y_test.cpu()
    from sklearn.metrics import classification_report
    print(f'{model_file}')
    print(classification_report(y_test, y_pred, target_names = classes))  
    
    corrctcount = wrongcount = 0
    for i , (target, pred) in enumerate(zip(y_test,y_pred)):
        # print(f'target:{bool(target)} , pred:{pred}')
        # if i > 100 : break
        
        if bool(target)==pred:
            corrctcount+=1
        else:
            wrongcount+=1
    
    print(f'corrctcount:{corrctcount}')
    print(f'wrongcount:{wrongcount}')
    print(f'Accuracy Test :{ (corrctcount/( corrctcount + wrongcount)) *100 }')
        
    del net

/tmp/ipykernel_158/146710385.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainToday[df.RainToday == 'No'] = 0
/tmp/ipykernel_158/146710385.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'Yes'] = 1
/tmp/ipykernel_158/146710385.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.RainTomorrow[df.RainTomorrow == 'No'] = 0
/opt/conda/envs/rapids/lib/python3.8/site-packages/sklearn/metr

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124689 entries, 0 to 145458
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Rainfall      124689 non-null  float64
 1   Humidity3pm   124689 non-null  float64
 2   RainToday     124689 non-null  int64  
 3   Pressure9am   124689 non-null  float64
 4   RainTomorrow  124689 non-null  int64  
dtypes: float64(3), int64(2)
memory usage: 5.7 MB
checkpoint_singleGPU.pt
              precision    recall  f1-score   support

     No rain       0.78      1.00      0.87     19361
        Rain       0.00      0.00      0.00      5577

    accuracy                           0.78     24938
   macro avg       0.39      0.50      0.44     24938
weighted avg       0.60      0.78      0.68     24938

corrctcount:19361
wrongcount:5577
Accuracy Test :77.63653861576711
